In [16]:
from smartiS import *
import ipywidgets as widgets
import sympy as sp
import numpy as np

from math import pi, sin, cos
from ipywidgets import HBox, VBox, Layout

class Impuls(smartiS):
    
    def __init__(self):
        
        # Ebenen definieren
        layers = ["masse1"]
        lx = 750
        ly = 650

        # smartiS-Klasse initialisieren
        super().__init__(canvases = layers,
                         width    = lx,
                         height   = ly)
        
        # Elemente erzeugen
       
        self.v1_slider = widgets.IntSlider(min = 1,
                                          max = 30,
                                          value = 1,
                                          description = r"$v1$")
        self.v1_slider.observe(self.draw_masse1, "value")

        
        self.m1_slider = widgets.FloatSlider(min = 1.0,
                                          max = 500.0,
                                            step=0.1,
                                          value = 2.0,
                                          description = r"$m1$")
        self.m1_slider.observe(self.draw_masse1, "value")
        
        self.m2_slider = widgets.FloatSlider(min = 1.0,
                                          max = 500.0,
                                           step=0.1,
                                          value = 2.0,
                                          description = r"$m2$")
        self.m2_slider.observe(self.draw_masse1, "value")
        
        self.v2_slider = widgets.IntSlider(min = 1,
                                          max = 30,
                                          value = 1,
                                          description = r"$v1$")
        self.v2_slider.observe(self.draw_masse1, "value")
        
        self.play1 = widgets.Play(
            value=0,
            min=0,
            max=299,
            step=1,
            interval=50,
            description="Starten",
            disabled=False
        )
        self.play1.observe(self.draw_masse1, "value")
        
        self.play2 = widgets.Play(
            value=1,
            min=1,
            max=300,
            step=1,
            interval=50,
            description="Starten",
            disabled=False
        )
        self.play2.observe(self.draw_masse1, "value")
        
        widgets.jslink((self.play1, 'value'),(self.play2, 'value'))

        self.auswahl_buttons = widgets.RadioButtons(options=['inelastisch', 'elastisch'],
                                                    value='inelastisch',
                                                    layout={'width': 'max-content'},
                                                    description='Stoßart:',
                                                    disabled=False
                                                   )
        self.auswahl_buttons.observe(self.draw_masse1, "value")

        # Elemente darstellen
        display(HBox([
            self.canvas,
            VBox([HBox([], layout=Layout(height="650px")),
                  self.auswahl_buttons,
                  self.m1_slider,
                  self.m2_slider,
                  self.v1_slider,
                  self.v2_slider,
                  self.play1,
                  self.play2
                 ])
        ]))
        
        self.draw_masse1()
    
    
    
    def draw_masse1(self,*args):
        
        # Masse1 zeichnen
        masse1 = self["masse1"]
        a = self.auswahl_buttons.value
        tA = self.play1.value
        tE = self.play2.value
        
        m1 = self.m1_slider.value
        v1A = self.v1_slider.value
        d1 = ((6 * m1)/pi)**(1/3) * 5
        x1 = 50 + v1A * tA
        x1_Stoß = np.zeros(300)
        for i in range (300):
            x1_Stoß[i]=50 + v1A * tA
        
        m2 = self.m2_slider.value
        v2A = self.v2_slider.value
        d2 = ((6 * m2)/pi)**(1/3) * 5
        x2 = 700 - v2A * tA
        x2_Stoß = np.zeros(300)
        for i in range (300):
            x2_Stoß[i]=50 + v1A * tA        
            
        
        '''
        def f(m1, m2, v1A, v2A):

            v1E, v2E = sp.symbols("v1E, v2E")

            equations = [
                sp.Eq(m1*v1A+m2*(-v2A) , m1*v1E+m2*(-v2E) ),
                sp.Eq(1/2*m1*v1A**2 +1/2*m2*(-v2A)**2, 1/2*m1*v1E**2+1/2*m2*(-v2E)**2 ),
                ]
            print(sp.solve(equations))
        
        '''
            
            
        #vor dem Stoß
        if (x1 + (d1)) < (x2 - (d2)):
            
            with hold_canvas(masse1):

                masse1.clear()
                masse1.stroke_arc(x1, 325, d1, 0, 2*pi)
                masse1.fill_style = "#0000ff"
                masse1.fill_arc(x1, 325, d1, 0, 2*pi)

                masse1.stroke_arc(x2, 325, d2, 0, 2*pi)
                masse1.fill_style = "#a000ff"
                masse1.fill_arc(x2, 325, d2, 0, 2*pi)

            
        #nach dem Stoß
        elif (x1 + (d1)) >= (x2 - (d2)):
            self.play1._playing = False
            
            #if statemant mit Radiobutton elastischer und inelastischer Stoß
            if a == 'inelastisch':
                vE = m1/(m1+m2) * v1A + m2/(m1+m2) * (-v2A)
                x1E = x1_Stoß[tA] + vE * tE
                x2E = x2_Stoß[tA] + vE * tE
            
                with hold_canvas(masse1):

                    masse1.clear()
                    masse1.stroke_arc(x1E, 325, d1, 0, 2*pi)
                    masse1.fill_style = "#0000ff"
                    masse1.fill_arc(x1E, 325, d1, 0, 2*pi)

                    masse1.stroke_arc(x2E, 325, d2, 0, 2*pi)
                    masse1.fill_style = "#a000ff"
                    masse1.fill_arc(x2E, 325, d2, 0, 2*pi)
                                
            
            elif a == 'elastisch':
                v1E = (m1 * v1A + m2 * (2*(-v2A) - v1A))/(m1 + m2)
                v2E = (m2 * (-v2A) + m1 * (2*v1A - (-v2A)))/(m1 + m2)
                x1E = x1_Stoß[tA] + v1E * tE
                x2E = x2_Stoß[tA] + v2E * tE
                
                with hold_canvas(masse1):

                    masse1.clear()
                    masse1.stroke_arc(x1E, 325, d1, 0, 2*pi)
                    masse1.fill_style = "#0000ff"
                    masse1.fill_arc(x1E, 325, d1, 0, 2*pi)

                    masse1.stroke_arc(x2E, 325, d2, 0, 2*pi)
                    masse1.fill_style = "#a000ff"
                    masse1.fill_arc(x2E, 325, d2, 0, 2*pi)

    
            
      
            
I = Impuls()
